# Data Science Part Time Course
## SQL Intro

<a name="demo"></a>
## Interacting with SQLite from Python

### The `sqlite3` package

The command line utility can be useful for basic SQL tasks, but since we're using python for the rest of code it will often be easier to access sqlite directly from within python.  We can use the python [`sqlite3`](https://docs.python.org/2.7/library/sqlite3.html) package for just this purpose.

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

The syntax to create a table is similar to the console, only now we use the `execute` method of the cursor object `c` that we just created:

In [2]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes
conn.commit()

OperationalError: table houses already exists

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitutionusing the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [3]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

In [4]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [5]:
c.execute('select * from houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2390, 4, 34, 319000),
 (6, 1870, 3, 14, 289000),
 (7, 1505, 3, 90, 269000),
 (8, 2104, 3, 70, 399900),
 (9, 1600, 3, 28, 329900),
 (10, 2400, 3, 44, 369000),
 (11, 1416, 2, 49, 232000),
 (12, 3000, 4, 75, 539900),
 (13, 1985, 4, 61, 299900),
 (14, 1534, 3, 12, 314900),
 (15, 1427, 3, 57, 198999),
 (16, 1380, 3, 14, 212000),
 (17, 1494, 3, 15, 242500),
 (18, 1940, 4, 7, 239999),
 (19, 2000, 3, 27, 347000),
 (20, 1890, 3, 45, 329999),
 (21, 4478, 5, 49, 699900),
 (22, 1268, 3, 58, 259900),
 (23, 2300, 4, 77, 449900),
 (24, 1320, 2, 62, 299900),
 (25, 1236, 3, 78, 199900),
 (26, 2609, 4, 5, 499998),
 (27, 3031, 4, 21, 599000),
 (28, 1767, 3, 44, 252900),
 (29, 1888, 2, 79, 255000),
 (30, 1604, 3, 13, 242900),
 (31, 1962, 4, 53, 259900),
 (32, 3890, 3, 36, 573900),
 (33, 1100, 3, 60, 249900),
 (34, 1458, 3, 29, 464500),
 (35, 2526, 3, 13, 469000),
 (36, 2200, 3, 28, 475000),
 (3




#### Adding data from a csv file
Next let's load our housing.csv data into an array, and then `INSERT` those records into the database.  In this example we'll use the numpy `genfromtxt` function to read the file and parse the contents. 

In [6]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = genfromtxt('datasets/housing-data.csv', dtype='i8', 
                    delimiter=',', skip_header=1).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

In [8]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

A reason for this example - remember that all elements in a numpy array must be the same data type, so if we want to 'add a None' to each row, we need to work around this.  Lists can contain mixed types, so that is one approach.

Still, in this case the value we're adding is the same for all records, so we could have simply used a 'None' in the INSERT statement directly.

**Check**: How do you delete data?

In [9]:
# similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# here results is a cursor object - use fetchall() to extract a list
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (5, 2390, 4, 34, 319000),
 (12, 3000, 4, 75, 539900),
 (13, 1985, 4, 61, 299900),
 (18, 1940, 4, 7, 239999),
 (23, 2300, 4, 77, 449900),
 (26, 2609, 4, 5, 499998),
 (27, 3031, 4, 21, 599000),
 (31, 1962, 4, 53, 259900),
 (40, 2040, 4, 75, 314900),
 (42, 1811, 4, 24, 285900),
 (45, 2132, 4, 28, 345000),
 (46, 4215, 4, 66, 549000),
 (47, 2162, 4, 43, 287000),
 (50, 2567, 4, 57, 314000),
 (53, 1852, 4, 64, 299900),
 (56, 2390, 4, 34, 319000),
 (63, 3000, 4, 75, 539900),
 (64, 1985, 4, 61, 299900),
 (69, 1940, 4, 7, 239999),
 (74, 2300, 4, 77, 449900),
 (77, 2609, 4, 5, 499998),
 (78, 3031, 4, 21, 599000),
 (82, 1962, 4, 53, 259900),
 (91, 2040, 4, 75, 314900),
 (93, 1811, 4, 24, 285900),
 (96, 2132, 4, 28, 345000),
 (97, 4215, 4, 66, 549000),
 (98, 2162, 4, 43, 287000),
 (101, 2567, 4, 57, 314000),
 (104, 1852, 4, 64, 299900),
 (107, 2390, 4, 34, 319000),
 (114, 3000, 4, 75, 539900),
 (115, 1985, 4, 61, 299900),
 (120, 1940, 4, 7, 239999),
 (125, 2300, 4, 77, 4

### Pandas connector

While databases provide many analytical capabilities, often it's useful to pull the data back into Python for more flexible programming. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales to present a report or dashboard, this operation is likely not changing and operating on a large dataset. This can run very efficiently in a database rather than by connecting to it with Python.

However, if we want to investigate login or sales data further and ask more interactive questions, then Python would be more practical.

In [ ]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

### Writing data into a database

Data in Pandas can be loaded into a relational database. For the most part, Pandas can use column information to infer the schema for the table it creates. For the next demo we will use the Rossmann stores dataset.

In [10]:
import pandas as pd

data = pd.read_csv('datasets/housing-data.csv', low_memory=False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [11]:
data.to_sql('houses_pandas',
            con=conn,
            if_exists='replace',
            index=False)

In [14]:
c.execute('select bdrms, avg(price) from houses_pandas group by bdrms').fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

In [ ]:
c.commit()

In [12]:
data

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


<a name="guided-practice"></a>


# SQL Syntax 


#### SELECT
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.
>
```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is some filter applied to the rows, where rows that match the condition will be in the output.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for when houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [SQLite - Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax